In [1]:
!pip install -U transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [3]:
def generate_prompt(new_sentence):
    prompt = (
        f"translate English to Indonesian: \n{new_sentence}"
    )

    return prompt

In [4]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# Ensure the nltk 'punkt' tokenizer is downloaded
nltk.download('punkt_tab')

def calculate_bleu_single_reference(reference_sentence, candidate_sentence):
    """
    Calculate the BLEU score for a single predicted translation against one reference.

    Parameters:
        reference_sentence (str): The correct reference translation.
        candidate_sentence (str): The predicted translation.

    Returns:
        float: The BLEU score.
    """
    # Tokenize the reference sentence and candidate sentence
    tokenized_reference = [nltk.word_tokenize(reference_sentence)]
    tokenized_candidate = nltk.word_tokenize(candidate_sentence)

    # Use smoothing to handle short sentences
    smoothing_function = SmoothingFunction().method1

    # Calculate BLEU score
    bleu_score = sentence_bleu(tokenized_reference, tokenized_candidate, smoothing_function=smoothing_function)
    return bleu_score

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [5]:
# Test Data Examples
# english_sentences =
#     "Can I help you?"
#     "As you can see, I'm jet-lagged and trying to adjust to the time-zone change."
#     "That's what the policeman just said."
#     "Yeah, we've been teaching him, but really, he's been teaching us."
#     "Someone wants to kill him."
#     "Hey Four, that's pretty good right?"
#     "You should've told me, boy."
#     "Your bed is in the corner, over there."
#     "Hey, kid, you've done really good so far, so what we're gonna do right now is we're gonna call your daddy."

# indonesian_targets =
#     "ada yang bisa kubantu?"
#     "Seperti yang kamu lihat , aku alami jet - lag dan mencoba untuk menyesuaikan diri dengan perubahan zona waktu ."
#     "Begitulah kata polisinya tadi."
#     "Ya, kami telah mengajari dia, tapi sungguh, ia telah mengajari kita."
#     "Seseorang ingin membunuhnya."
#     "Four, Tak begitu buruk, kan?"
#     "Kau harusnya katakan padaku, Nak."
#     "Ranjangmu disudut itu."
#     "Hei, nak, sejauh ini kau telah melakukannya dengan baik sekali, jadi apa yang akan kita lakukan sekarang adalah kita akan menelpon ayahmu."

# The new sentence to translate
new_sentence = "Yeah, we've been teaching him, but really, he's been teaching us."

# Generate the prompt
prompt = generate_prompt(new_sentence)
print(prompt)

translate English to Indonesian: 
Yeah, we've been teaching him, but really, he's been teaching us.


In [6]:
# Tokenize the input prompt
inputs = tokenizer(prompt, return_tensors="pt", max_length=100, truncation=True)

outputs = model.generate(**inputs, max_length=128)

# Decode the output
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(response)

Ja, wir l'avons predat, aber wirklich, er l'a predat.


#Fine Tuned

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
path = "/content/drive/MyDrive/Kuliah/Semester 7/NLP/T5/"

model_path = path + "t5_small_finetuned_100k"

In [9]:
from transformers import TFT5ForConditionalGeneration

fine_tuned_model = TFT5ForConditionalGeneration.from_pretrained(model_path)

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at /content/drive/MyDrive/Kuliah/Semester 7/NLP/T5/t5_small_finetuned_100k.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [10]:
# Tokenize the input prompt
input_ids = tokenizer(prompt, return_tensors="tf", max_length=512, truncation=True).input_ids

outputs = fine_tuned_model.generate(input_ids, max_length=128, num_beams=5, early_stopping=True)

# Decode the output
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(response)

Ya, kita telah mendapatkan dia, tapi benar-benar, dia mendapatkan kita.


In [11]:
# Test Data Examples
# english_sentences =
#     "Can I help you?"
#     "As you can see, I'm jet-lagged and trying to adjust to the time-zone change."
#     "That's what the policeman just said."
#     "Yeah, we've been teaching him, but really, he's been teaching us."
#     "Someone wants to kill him."
#     "Hey Four, that's pretty good right?"
#     "You should've told me, boy."
#     "Your bed is in the corner, over there."
#     "Hey, kid, you've done really good so far, so what we're gonna do right now is we're gonna call your daddy."

# indonesian_targets =
#     "ada yang bisa kubantu?"
#     "Seperti yang kamu lihat , aku alami jet - lag dan mencoba untuk menyesuaikan diri dengan perubahan zona waktu ."
#     "Begitulah kata polisinya tadi."
#     "Ya, kami telah mengajari dia, tapi sungguh, ia telah mengajari kita."
#     "Seseorang ingin membunuhnya."
#     "Four, Tak begitu buruk, kan?"
#     "Kau harusnya katakan padaku, Nak."
#     "Ranjangmu di sudut itu."
#     "Hei, nak, sejauh ini kau telah melakukannya dengan baik sekali, jadi apa yang akan kita lakukan sekarang adalah kita akan menelpon ayahmu."

# Single reference translation
reference_sentence = "Ya, kami telah mengajari dia, tapi sungguh, ia telah mengajari kita."
candidate_sentence = "Ya, kita telah mendapatkan dia, tapi benar-benar, dia mendapatkan kita."

# Calculate BLEU score
bleu_score = calculate_bleu_single_reference(reference_sentence, candidate_sentence)
print(f"BLEU Score: {bleu_score:.4f}")

BLEU Score: 0.1030


In [12]:
from datasets import load_dataset
import pandas as pd

In [20]:
def load_test_data():
    dataset = load_dataset("Helsinki-NLP/opus-100", "en-id")
    test_data = dataset["test"]
    return test_data

In [14]:
def evaluate_model(fine_tuned_model, tokenizer, test_data, output_path):
    results = []
    total_bleu_score = 0
    count = 0

    for example in test_data:
        # Extract input and target from the dataset
        english_sentence = example["translation"]["en"]
        reference_sentence = example["translation"]["id"]

        # Generate the prompt
        prompt = generate_prompt(english_sentence)

        # Tokenize and generate prediction
        input_ids = tokenizer(prompt, return_tensors="tf", max_length=512, truncation=True).input_ids
        outputs = fine_tuned_model.generate(input_ids, max_length=128, num_beams=5, early_stopping=True)
        prediction_sentence = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Calculate BLEU score
        bleu_score = calculate_bleu_single_reference(reference_sentence, prediction_sentence)
        total_bleu_score += bleu_score
        count += 1

        # Store results
        results.append({
            "Input (English)": english_sentence,
            "Reference (Indonesian)": reference_sentence,
            "Prediction (Indonesian)": prediction_sentence,
            "BLEU Score": bleu_score,
        })

    average_bleu_score = total_bleu_score / count if count > 0 else 0

    # Save results to Excel
    results_df = pd.DataFrame(results)
    results_df.to_excel(output_path, index=False)
    print(f"Results saved to {output_path}")
    print(f"Average BLEU Score: {average_bleu_score:.4f}")

    return results_df

In [21]:
# Paths
output_file = "/content/drive/MyDrive/Kuliah/Semester 7/NLP/T5/t5_test_results.xlsx"

# Load test data
test_data = load_test_data()

test_data[0]

{'translation': {'en': 'He was on the run.',
  'id': 'Dia sedang dalam pelarian.'}}

In [22]:
# Evaluate the model on the test data and save results
results_df = evaluate_model(fine_tuned_model, tokenizer, test_data, output_file)

Results saved to /content/drive/MyDrive/Kuliah/Semester 7/NLP/T5/t5_test_results.xlsx
Average BLEU Score: 0.1537


In [23]:
results_df.head(10)

,Input (English),Reference (Indonesian),Prediction (Indonesian),BLEU Score
0,He was on the run.,Dia sedang dalam pelarian.,Dia adalah pergi.,0.074410
1,"How d'ya like that, Spidey?","Bagaimana sekarang, Spidey?","Bagaimana seperti itu, Spidey?",0.217119
2,"- Staff Sergeant, sir!","- Staf Sersan, Pak!","- Sersan, Pak!",0.578930
3,I'll be with her.,Aku akan bersamanya.,Aku akan bersama dia.,0.125743
4,"Because of us, they will be ruined.","Karena kita, mereka akan hancur.","Karena kita, mereka akan terlalu tahu.",0.541082
5,- Now sit you down.,- Sekarang coba duduk.,- Sekarang membuatmu.,0.146438
6,"Okay, listen to me, Cole.","Oke, dengarkan aku, Cole.","Oke, dengarkan aku, Cole.",1.000000
7,- You're going to hell!,Kau akan masuk Neraka!,- Kau akan ke adalah!,0.095544
8,Then perhaps I can be of assistance in a diffe...,Mungkin aku bisa menolongamu dengan cara lain.,Maafkan aku bisa menjadi asisten di mana yang ...,0.049394
9,For the little kids.,Untuk anak-anak kecil.,Untuk anak-anak kecil.,1.000000
